In [470]:
import pandas as pd

Let's do some preliminary analysis.

### sales
1. dimensions of the dataset;
2. data cleaning 	
	1. remove the empty columns and rows;
	2. check for null columns (each entry is `NaN`) / check for null rows;
	3. check types;
	4. `date`:
		- check range;
	5. `is_open`:
		- check is composed by `YES` and `NO` only;
		- edit the exeptions; 
	6. `sales_amount`: 
		- impose `if is_open == "NO" then sales_amount = 0` (and fix exeptions);
		- check if all values are non-negative (NO!);
		- edit negatives and `NaN`;

### market
1. dimensions of the dataset;
2. data cleaning 	
	1. remove the empty columns and rows;
	2. check for null columns (each entry is `NaN`) / check for null rows;
	3. check types;
	4. `id`:
		- unique identifier;
	5. `market_type`:
		- solve `NaN` values; 
	6. `squere_feet`: 
		- check positive; 
	7. `avg_customer`: 
		- check positive (and correct);
		- check integer (and correct);
	8. `competitor_distance`: 
		- - check positive;
	9. `has_promotion`:
		- check is composed by `YES` and `NO` only;
		- edit the exeptions;

# sales

In [471]:
sales = pd.read_csv("data/sales.csv")
display(sales)

,market_id,date,is_open,sales_amount,
0,XC14,31/12/2022,YES,55727.34,NaN
1,WG25,31/12/2022,YES,12984.10,NaN
2,UX72,31/12/2022,YES,13046.56,NaN
3,EE84,31/12/2022,YES,60916.76,NaN
4,RF54,31/12/2022,YES,12376.42,NaN
...,...,...,...,...,...
523944,ZM74,01/01/2021,YES,21553.84,NaN
523945,WW60,01/01/2021,YES,56371.18,NaN
523946,LU14,01/01/2021,YES,24107.19,NaN
523947,HY57,01/01/2021,YES,12257.38,NaN


## 1

dimensions of the dataset

In [472]:
n_rows_sales = sales.shape[0]
n_cols_sales = sales.shape[1]
print("the number of rows is " + str(n_rows_sales) + " while the number of columns is " + str(n_cols_sales))

the number of rows is 523949 while the number of columns is 5


## 2

### 2.1

In [473]:
# remove empty colums
sales = sales.dropna(axis = 1, how = "all")
n_cols_sales = sales.shape[1]

# remove empty rows
sales = sales.dropna(axis = 0, how = "all")
n_rows_sales = sales.shape[0]
	
display(sales)

,market_id,date,is_open,sales_amount
0,XC14,31/12/2022,YES,55727.34
1,WG25,31/12/2022,YES,12984.10
2,UX72,31/12/2022,YES,13046.56
3,EE84,31/12/2022,YES,60916.76
4,RF54,31/12/2022,YES,12376.42
...,...,...,...,...
523944,ZM74,01/01/2021,YES,21553.84
523945,WW60,01/01/2021,YES,56371.18
523946,LU14,01/01/2021,YES,24107.19
523947,HY57,01/01/2021,YES,12257.38


### 2.2

check for NaN

In [474]:
n_nan = sales.isnull().sum().sum()
print("the total number of NaN is " + str(n_nan) + "\n")

print("the number of NaN in each column of sales is:")
for index in range(0, n_cols_sales):
	col = sales.iloc[:, index]
	n_nan_col = col.isnull().sum().sum()
	print("- " + sales.columns[index] + " = " + str(n_nan_col))

the total number of NaN is 14

the number of NaN in each column of sales is:
- market_id = 0
- date = 0
- is_open = 0
- sales_amount = 14


(commento) siccome i NaN sono solo in sales_amount bisogna capire come trattarli

### 2.3


check types

In [475]:
sales.dtypes

market_id        object
date             object
is_open          object
sales_amount    float64
dtype: object

### 2.4

check range of date

In [476]:
sales["date"] = pd.to_datetime(sales["date"], format = "%d/%m/%Y")
start_date = "01/01/2021"
end_date = "31/12/2022"

# boolean panda's series: True if the date is acceptable (inside the range), False otherwise
check_date_range = sales["date"].between(start_date, end_date)

n_date_out_of_range = n_rows_sales - check_date_range.sum()
print("the number of dates out of range is " + str(n_date_out_of_range))

the number of dates out of range is 0


### 2.5

check is_open is compesed only by yes and no

In [477]:
# boolean panda's series: True if is_open is acceptable ("YES" or "NO"), False otherwise
check_is_open = sales[~sales["is_open"].isin(["YES", "NO"])].index
print("the rows with no YES or NO in is_open are: " + str(list(check_is_open)))

display(pd.DataFrame(sales["is_open"][check_is_open]))

the rows with no YES or NO in is_open are: [49154]


,is_open
49154,YESS


(commento)

In [478]:
sales.loc[check_is_open[0], "is_open"] = "YES"

### 2.6

controlliamo che i NO abbiano sales_amount uguale a 0 e li correggiamo

In [479]:
# sales dataframe restricted to the condition is_open == "NO" with non-acceptable sales_amounts (!= 0)
sales_amount_no_error = sales[(sales["is_open"] == "NO") & (sales["sales_amount"] != 0)]
print("indexes where is_open is NO and sales_amount is not 0: " + str(list(sales_amount_no_error.index)))
sales.loc[sales_amount_no_error.index, "sales_amount"] = 0

indexes where is_open is NO and sales_amount is not 0: [753]


In [480]:
# sales dataframe restricted to the condition sales_amounts < 0
sales_amount_negative = sales[sales["sales_amount"] < 0]
sales_amount_negative

,market_id,date,is_open,sales_amount
47528,XC14,2022-10-16,YES,-999999.0
48150,XC14,2022-10-15,YES,-999999.0
48772,XC14,2022-10-14,YES,-999999.0
49394,XC14,2022-10-13,YES,-999999.0
50016,XC14,2022-10-12,YES,-999999.0
50638,XC14,2022-10-11,YES,-999999.0


trattiamo i dati negativi e NaN allo stesso modo: sostituiamo il valore con la media dei sales_amount dello stesso market <br>
(commento: altra opzione sarebbe mettere media sullo stesso giorno con market dello stesso tipo nello stessa nazione)

In [481]:
# sales dataframe restricted to the rows with non-acceptable sales_amounts (NaN or negative values)
sales_amount_error = sales[sales["sales_amount"].isnull() | (sales["sales_amount"] < 0)]
display(sales_amount_error)

# sales dataframe restricted to the condition is_open == "NO"
sales_amount_no = sales[(sales["is_open"] == "NO")]

# (unique) market_id's of the rows associated with errors in sales_amount
market_id_error = pd.unique(sales_amount_error["market_id"])
# initialization of the dataframe containing the market_id's and their correspondent mean sales_amount
mean_sales_amount = pd.DataFrame()
for market in market_id_error:
	# sales dataframe restrected to the rows with market_id = market
	market_sales_amount = (sales[sales["market_id"] == market]["sales_amount"])
	# indexes of the rows of sales dataframe, restricted to market_id = market and sales_amount > 0
	correct_indexes = (sales[(sales["sales_amount"] > 0) & (sales["market_id"] == market)]).index
	# row containing market_id its correspondent mean sales_amount (computed considering only positive sales_amount)
	new_row = pd.DataFrame({'market_id': [market], 'sales_amount': [sales.loc[correct_indexes, "sales_amount"].mean()]})
	mean_sales_amount = pd.concat([mean_sales_amount, new_row])

# correcting errors:
# we iterate over the non-acceptable rows of the dataframe, subsituting the wrong sales_amounts with the previously computed mean
for index, row in (sales.loc[sales_amount_error.index]).iterrows():
	sales.loc[index, "sales_amount"] = mean_sales_amount[mean_sales_amount["market_id"] == row["market_id"]].loc[0, "sales_amount"]
sales.loc[sales_amount_error.index]

,market_id,date,is_open,sales_amount
98,OG21,2022-12-31,YES,NaN
47528,XC14,2022-10-16,YES,-999999.0
48150,XC14,2022-10-15,YES,-999999.0
48772,XC14,2022-10-14,YES,-999999.0
49394,XC14,2022-10-13,YES,-999999.0
50016,XC14,2022-10-12,YES,-999999.0
50638,XC14,2022-10-11,YES,-999999.0
174088,UF73,2022-04-12,YES,NaN
223239,BY57,2022-02-05,YES,NaN
509998,NL22,2021-01-19,YES,NaN


,market_id,date,is_open,sales_amount
98,OG21,2022-12-31,YES,62768.862528
47528,XC14,2022-10-16,YES,60603.900400
48150,XC14,2022-10-15,YES,60603.900400
48772,XC14,2022-10-14,YES,60603.900400
49394,XC14,2022-10-13,YES,60603.900400
50016,XC14,2022-10-12,YES,60603.900400
50638,XC14,2022-10-11,YES,60603.900400
174088,UF73,2022-04-12,YES,12005.331014
223239,BY57,2022-02-05,YES,12628.222322
509998,NL22,2021-01-19,YES,12017.349917


# market

In [482]:
market = pd.read_csv("data/market.csv")
display(market)

,id,country,Unnamed: 2,market_type,square_feet,avg_customers,competitor_distance,has_promotions
0,XC14,SPAIN,NaN,HYPER,11135,773.0,515,YES
1,WG25,FRANCE,NaN,MINI,1402,252.0,522,YES
2,UX72,FRANCE,NaN,MINI,1452,265.0,15939,YES
3,EE84,SPAIN,NaN,HYPER,15259,874.0,754,YES
4,RF54,FRANCE,NaN,MINI,2497,191.0,34007,YES
...,...,...,...,...,...,...,...,...
745,ZM74,ITALY,NaN,SUPER,5778,356.0,4424,YES
746,WW60,FRANCE,NaN,HYPER,10923,907.0,41359,YES
747,LU14,ITALY,NaN,SUPER,5742,397.0,2841,YES
748,HY57,ITALY,NaN,MINI,2495,228.0,3848,YES


## 1

dimensions of the dataset

In [483]:
n_rows_market = market.shape[0]
n_cols_market = market.shape[1]
print("the number of rows is " + str(n_rows_market) + " while the number of columns is " + str(n_cols_market))

the number of rows is 750 while the number of columns is 8


## 2

### 2.1

In [484]:
# remove empty colums
market = market.dropna(axis = 1, how = "all")
n_cols_market = market.shape[1]

# remove empty rows
sales = market.dropna(axis = 0, how = "all")
n_rows_market = market.shape[0]
	
display(market)

,id,country,market_type,square_feet,avg_customers,competitor_distance,has_promotions
0,XC14,SPAIN,HYPER,11135,773.0,515,YES
1,WG25,FRANCE,MINI,1402,252.0,522,YES
2,UX72,FRANCE,MINI,1452,265.0,15939,YES
3,EE84,SPAIN,HYPER,15259,874.0,754,YES
4,RF54,FRANCE,MINI,2497,191.0,34007,YES
...,...,...,...,...,...,...,...
745,ZM74,ITALY,SUPER,5778,356.0,4424,YES
746,WW60,FRANCE,HYPER,10923,907.0,41359,YES
747,LU14,ITALY,SUPER,5742,397.0,2841,YES
748,HY57,ITALY,MINI,2495,228.0,3848,YES


### 2.2

check for NaN

In [485]:
n_nan = market.isnull().sum().sum()
print("the total number of NaN is " + str(n_nan) + "\n")

print("the number of NaN in each column of sales is:")
for index in range(0, n_cols_market):
	col = market.iloc[:, index]
	n_nan_col = col.isnull().sum().sum()
	print("- " + market.columns[index] + " = " + str(n_nan_col))

the total number of NaN is 3

the number of NaN in each column of sales is:
- id = 0
- country = 0
- market_type = 2
- square_feet = 0
- avg_customers = 1
- competitor_distance = 0
- has_promotions = 0


### 2.3

In [486]:
market.dtypes

id                      object
country                 object
market_type             object
square_feet              int64
avg_customers          float64
competitor_distance      int64
has_promotions          object
dtype: object

c'è un problema: avg_customers è float, invece noi lo si vuole int. Gli altri tipi vanno bene